In [4]:
def parse_input(filepath):
    with open(filepath, 'r') as f:
        lines = [l.strip() for l in f]
    return lines

In [144]:
lines = parse_input('input.txt')

In [145]:
lines[:10]

['{<[[<<<[[{[[<<()()>>[([]<>){{}{}}]]]<(<{{}<>}{{}{}}><{<>[]}[{}{}]>)({(())}<[(){}][(){}]>)>}]<<<(<<{}',
 '{[({({([({(((<[]()>[()<>]){[<>[]](<>[])})<<<[]()}>(<()()>)>)}[{((<{}[]>{<>()}){[<>{}]<<><>>})(({[]()}<<>()>)<',
 '(((([{{{<{<[[<()<>><<>{}>]]{<(<>())>}>{<[([]<>)<{}{}>][[<><>][<>[]]]>[{(<>{})[<>()]}<<()[]>[[]<',
 '<<[<([[{{{[({<<>()>}<<<>[]>([]())>){{{{}[]}<(){}>)}]{{[(()())[(){}]]<<[]{}>({}[])>}({<<><>>}<([]<>){()[]}>)}',
 '<[(({[({<<[[[{<>{}}(<><>))<(<>[])[{}[]]>]<[([][])[(){}]]>]([<[()()](<>[])>]<(<{}()>[{}<>])({<>[',
 '(<[[{[[[(<((<<{}{}><[]<>>><[{}<>][<>[]]>)<(<{}<>><{}{}>)[{()[]}<<>()>]>)>[(<{({}{})}[{()[]}{[]<>}]>){<(([]',
 '(([[({{{(<([({[]{}}){{[][]]}])>)[<([[[{}]{(){}}][([]())]]([{[][]}{<>()}](({}()){<>{}})))[{({',
 '{(<<([([(<<[{((){})([]<>)}]<{{{}{}}{{}[]}}<{(){}}{()[]})>><([<[]{}>([]<>)](<()<>>)){<([]<>',
 '{[<<[<{{((({<[[]{}]{{}}>{<{}<>>[<>[]]}})<{[(<><>><{}<>>][{[]{}}]}[[[[][]]<<>{}>]{[[]()][[]<>]}]>)[(',
 '({<(<([{<{<[(((){}){(){}})]>}[{{{{()()}<[]()>}{{<>()}<{}{}>

In [146]:
bracket_pairs = [
    ('()', 3, 1),
    ('{}', 1197, 3),
    ('<>', 25137, 4),
    ('[]', 57, 2)
]

In [147]:
close_to_open = {k: v for (v, k), *_ in bracket_pairs}

In [148]:
open_to_close = {k: v for (k, v), *_ in bracket_pairs}

In [149]:
corrupt_scores_dict = {k: v for (_, k), v, _ in bracket_pairs}

In [150]:
incomplete_scores_dict = {k: v for (_, k), _, v in bracket_pairs}

In [151]:
brackets, corrupt_scores, incomplete_scores = list(zip(*bracket_pairs))

In [152]:
opening_brackets, closing_brackets = list(zip(*brackets))

In [153]:
def find_error(s):
    stack = list()
    for e in s:
        if e in open_brackets:
            stack.append(e)
        elif e in closing_brackets:
            if not stack:
                return ('corrupt', e)
            elif bracket_dict[e] != stack.pop():
                return ('corrupt', e)
    
    closing_chars = list()
    while stack:
        closing_chars.append(open_to_close[stack.pop()])
    
    closing_string = ''.join(closing_chars)

    return ('incomplete', closing_string)

In [154]:
from functools import reduce

In [155]:
help(reduce)

Help on built-in function reduce in module _functools:

reduce(...)
    reduce(function, sequence[, initial]) -> value
    
    Apply a function of two arguments cumulatively to the items of a sequence,
    from left to right, so as to reduce the sequence to a single value.
    For example, reduce(lambda x, y: x+y, [1, 2, 3, 4, 5]) calculates
    ((((1+2)+3)+4)+5).  If initial is present, it is placed before the items
    of the sequence in the calculation, and serves as a default when the
    sequence is empty.



In [156]:
def get_incomplete_score(s):
    char_scores = (incomplete_scores_dict[e] for e in s)
    return reduce(lambda x, y: (5*x)  + y, char_scores, 0)

In [157]:
def get_score(errors):
    corrupt_score = 0
    incomplete_scores = []
    for error_type, s in errors:
        if error_type == 'corrupt':
            corrupt_score += corrupt_scores_dict[s]
        elif error_type == 'incomplete':
            incomplete_scores.append(get_incomplete_score(s))
    
    incomplete_score = sorted(incomplete_scores)[len(incomplete_scores)//2]

    return corrupt_score, incomplete_score

In [158]:
lines[0]

'{<[[<<<[[{[[<<()()>>[([]<>){{}{}}]]]<(<{{}<>}{{}{}}><{<>[]}[{}{}]>)({(())}<[(){}][(){}]>)>}]<<<(<<{}'

In [159]:
errors = [find_error(s) for s in lines]

In [160]:
errors[:10]

[('incomplete', '>>)>>>]>>>]]>}'),
 ('corrupt', '}'),
 ('incomplete', '>]>]}}>}}}]))))'),
 ('corrupt', ')'),
 ('corrupt', ')'),
 ('incomplete', '))>}])]]]}]]>)'),
 ('corrupt', ']'),
 ('corrupt', ')'),
 ('corrupt', '>'),
 ('corrupt', '>')]

In [163]:
corrupt_score, incomplete_score = get_score(errors)

In [164]:
corrupt_score, incomplete_score

(266301, 3404870164)